In [1]:
import numpy as np
import pandas as pd

from pathlib import Path

from sklearn.model_selection import KFold, train_test_split 
from scipy.stats import ks_2samp
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import KBinsDiscretizer,StandardScaler
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

In [2]:
data_dir = Path('./data_csv/')

data = {}
for file in data_dir.iterdir():
    name = file.stem.split('_')[1]
    data[name]  = pd.read_csv(file.as_posix())

In [3]:
x_cols = ['TrackP', 'TrackEta', 'NumLongTracks']
y_cols = ['RichDLLbt', 'RichDLLk', 'RichDLLmu', 'RichDLLp', 'RichDLLe']

One can see there's a peak of outliers at the left handside of the plot. Let's not bother about it so far.

In [4]:
def score_func(X, Y, Y_pred, n_slices=100):
    score = 0
    
    
    reference = pd.concat([X, Y], axis=1).values
    prediction = pd.concat([X,Y_pred], axis=1).values
    
    w_normal = np.random.normal(size=(n_slices, reference.shape[1]))
    
    
    for k in range(n_slices):
        score = max(score,
                    ks_2samp(
                        np.sum(w_normal[k] * reference, axis=1),
                        np.sum(w_normal[k] * prediction, axis=1)
                    )[0]
                   )
    return score



def cross_val(X, Y, Model, **kwargs):
    kf = KFold(n_splits=5)

    model = Model(**kwargs)
    model_scores = []

    for train_index, test_index in kf.split(X,Y):
        X_train = X.iloc[train_index]
        Y_train = Y.iloc[train_index]
        X_test  = X.iloc[test_index ]
        Y_test  = Y.iloc[test_index ]
  
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
  
        model_scores.append(score_func(X_test, Y_test, Y_pred))
    
    return model_scores

def compare_hists(Y_test, Y_pred, name):
    fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(8, 20))
    i = 0
    for col in Y_pred.columns:
        _, bins, _ = axes[i].hist(Y_test[col], bins=100 , label='test'      )
        _, _   , _ = axes[i].hist(Y_pred[col], bins=bins, label='prediction', alpha=0.7)
        axes[i].legend()
        axes[i].set_xlabel("%s: %s" % (name, col))
        i += 1
    fig.show();
    



##  Conditional Gaussian Model

$P(Y|X)\sim N(\mu(X),\Sigma(X))$

First we go for model with $Cov(y_i,y_j)=0, i\not =j$, namely distinct model for each component of $Y$. Also for simplicity we condition on only one component of $X$


In [5]:
X = data['pion'][x_cols]
Y = data['pion'][y_cols]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

Независимо бьем каждый из предикторов по квантилям. Для каждого "бина" считаем условное среднее/медиану и стандартное отклонение. model_outliers=True пытается учитывать экзит коды: Смотрим где все таргеты нулевые или -999, считаем долю таких событий, при генерации сначала сэмплим бернули на наличие выброса, потом сэмплим или нормально или снова бернули на тип выброса (0 или -999). Также не используем их при вычислении бинов.

In [6]:
class Model_1:
    def __init__(self, n_bins=7, model_outliers=False):
        self.n_bins = n_bins
        self.model_outliers = model_outliers
 
    def fit(self, X, Y):
        
        X = X.copy()
        Y = Y.copy()
        
        self.x_cols = X.columns.tolist()
        self.y_cols = Y.columns.tolist()

        
        if self.model_outliers:
            mask1 = (Y == -999).values.all(axis=1)
            mask2 = (Y == 0).values.all(axis=1)
            mask = (mask1 | mask2)
        
            Y = Y[~mask]
            X = X[~mask] 

            self.probs1 = mask.mean()
            self.probs2 = mask1.mean()/mask.mean()
        
        self.binarizer = KBinsDiscretizer(n_bins=self.n_bins, encode='onehot-dense')
        
    
        
        self.ohe_cols = ['{}_{}'.format(a, b+1) 
                for a,b 
                in zip(np.repeat(self.x_cols,self.n_bins), np.tile(range(self.n_bins), len(self.x_cols)))
                ] 
        
        labels = self.binarizer.fit_transform(X)
        labels = pd.DataFrame(labels, columns = self.ohe_cols, dtype=np.int8,index=X.index)
        
        train = pd.concat([labels, Y], axis=1)
        
        self.gmean = Y.mean()
        self.gstd = Y.std()
        
        self.means = train.groupby(self.ohe_cols,as_index=False).mean()
        self.stds = train.groupby(self.ohe_cols, as_index=False).std(ddof=0)

    def predict(self, X):
        
        labels = self.binarizer.transform(X)
        labels = pd.DataFrame(labels, columns=self.ohe_cols, dtype=np.int8, index=X.index)
        
        means = pd.merge(labels, self.means, how='left')[self.y_cols].fillna(self.gmean)
        stds = pd.merge(labels, self.stds, how='left')[self.y_cols].fillna(self.gstd)
        

        pred = np.random.normal(loc=means, scale=stds)
        if self.model_outliers:
            step1 = np.random.binomial(1, self.probs1,(len(X),1))
            step2 = np.random.binomial(1, self.probs2, (len(X),1))
            pred = -999*step1*step2 + (1-step1)*pred
        
        return pd.DataFrame(pred, columns = self.y_cols, index=X.index)

Здесь тоже самое, но для каждого бина оцениваем характерное значение предиктров(среднее/медиана по бину) и строим линейную регрессию на на среднее и стандартное отклонение таргетов. На предикте уже не смотрим в какой бин попал пример.

In [8]:
class Model_2:
    def __init__(self, n_bins=7, model_outliers=False):
        self.n_bins = n_bins
        self.model_outliers= model_outliers

    def fit(self, X, Y):
        
    
        self.x_cols = X.columns.tolist()
        self.y_cols = Y.columns.tolist()

        if self.model_outliers:
            mask1 = (Y == -999).values.all(axis=1)
            mask2 = (Y == 0).values.all(axis=1)
            mask = (mask1 | mask2)
        
            Y = Y[~mask]
            X = X[~mask] 

            self.probs1 = mask.mean()
            self.probs2 = mask1.mean()/mask.mean()
        
        self.binarizer = KBinsDiscretizer(n_bins=self.n_bins, encode='onehot-dense')
        self.ohe_cols = ['{}_{}'.format(a, b+1) 
                for a,b 
                in zip(np.repeat(self.x_cols,self.n_bins), np.tile(range(self.n_bins), len(self.x_cols)))
                ] 
        
        labels = self.binarizer.fit_transform(X)
        labels = pd.DataFrame(labels, columns = self.ohe_cols, dtype=np.int8, index=X.index)
        
        train = pd.concat([labels,X, Y], axis=1)
        
        y_means = train.groupby(self.ohe_cols, as_index=False).median()[self.y_cols].fillna(Y.median())
        y_stds = train.groupby(self.ohe_cols, as_index=False).std(ddof=0)[self.y_cols].fillna(Y.std())
        self.x_medians = train.groupby(self.ohe_cols, as_index=False)[self.x_cols].median().fillna(X.median())
        

        self.mean_regr = LinearRegression(normalize=True, n_jobs=-1)
        self.std_regr = LinearRegression(normalize=True, n_jobs=-1)
        
        self.mean_regr.fit(self.x_medians[self.x_cols], y_means)
        self.std_regr.fit(self.x_medians[self.x_cols], y_stds)
        
    def predict(self, X):
        
        
        means = self.mean_regr.predict(X)
        stds = self.std_regr.predict(X)

        pred = np.random.normal(loc=means, scale=np.maximum(0,stds))
        
        if self.model_outliers:
            step1 = np.random.binomial(1, self.probs1,(len(X),1))
            step2 = np.random.binomial(1, self.probs2, (len(X),1))
            pred = -999*step1*step2 + (1-step1)*pred
        
        
        
        return pd.DataFrame(pred, columns = self.y_cols, index=X.index)

In [30]:
m  = Model_1(15)
m.fit(X_train, Y_train)
Y_pred = m.predict(X_test)
score_func(X_test,Y_test, Y_pred)

0.030775753793969685

In [31]:
m  = Model_1(15,True)
m.fit(X_train, Y_train)
Y_pred = m.predict(X_test)
score_func(X_test,Y_test, Y_pred)

0.10983912128702966

In [32]:
m  = Model_2(15)
m.fit(X_train, Y_train)
Y_pred = m.predict(X_test)
score_func(X_test,Y_test, Y_pred)

0.4049607603139175

In [33]:
m  = Model_2(15, True)
m.fit(X_train, Y_train)
Y_pred = m.predict(X_test)
score_func(X_test,Y_test, Y_pred)

0.08030735754113966

Вообще из-за того что разбиваем на бины независимо по предикторам, то мы можем получать бины  из 1-2 элементов. 
Есть идея кластеризовать с помощью k-means в 3х мерном пространстве $X$ и использовать кластеры как бины. 

In [15]:
class Model_3():
    def __init__(self, n_bins=7, model_outliers=False):
        self.n_bins = n_bins
        self.model_outliers= model_outliers
    
    def fit(self, X, Y):
        
        self.y_cols = Y.columns
        
        if self.model_outliers:
            mask1 = (Y == -999).values.all(axis=1)
            mask2 = (Y == 0).values.all(axis=1)
            mask = (mask1 | mask2)
        
            Y = Y[~mask]
            X = X[~mask] 

            self.probs1 = mask.mean()
            self.probs2 = mask1.mean()/mask.mean() 
            
        kmeans = KMeans(n_clusters=self.n_bins, n_jobs=-1)
        scaler = StandardScaler()
        
        X_scaled = scaler.fit_transform(X)
        
        labels = kmeans.fit_predict(X_scaled)
        train = Y.copy()
        train['labels']  = labels
        means = train.groupby('labels').mean()
        stds = train.groupby('labels').std()
        
        self.kmeans = kmeans
        self.scaler = scaler
        self.means = means
        self.stds = stds
        
    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        labels = self.kmeans.predict(X_scaled)
        
        means = self.means.iloc[labels]
        stds = self.stds.iloc[labels]
        
        pred = np.random.normal(loc=means, scale=stds)
        if self.model_outliers:
            step1 = np.random.binomial(1, self.probs1,(len(X),1))
            step2 = np.random.binomial(1, self.probs2, (len(X),1))
            pred = -999*step1*step2 + (1-step1)*pred
        return pd.DataFrame(pred, columns = self.y_cols,index=X.index)
        

In [34]:
m = Model_3(n_bins=50,model_outliers=True)
m.fit(X_train, Y_train)
Y_pred = m.predict(X_test)

score_func(X_test, Y_test, Y_pred)

0.0862433100535196

In [18]:
scores = {}
for n_bins in [5,10,15,20,25]:
    scores[n_bins] = np.mean(cross_val(X,Y,Model_1, n_bins=n_bins, model_outliers=True))

In [19]:
scores

{5: 0.03374179307141685,
 10: 0.03610774285705175,
 15: 0.030001823911141246,
 20: 0.04944872500665249,
 25: 0.031101817246186214}

In [35]:
scores = {}
for n_bins in [5,10,15,20,25]:
    scores[n_bins] = np.mean(cross_val(X,Y,Model_2, n_bins=n_bins, model_outliers=True))

scores

{5: 0.06803873698631509,
 10: 0.08455765998170012,
 15: 0.0832966786216069,
 20: 0.10715258822705889,
 25: 0.09667968625156874}

In [20]:
model_scores = {}
for p in data:
    X = data[p][x_cols]
    Y = data[p][y_cols]
    model_scores[p] = cross_val(X,Y, Model_1, n_bins=15, model_outliers=True)

In [29]:
for p in model_scores:
    print(f'{p}:\t',np.mean(model_scores[p]))

muon:	 0.04945997345013275
ghost:	 0.055964279821399135
kaon:	 0.011767075330541613
proton:	 0.01032406151040649
pion:	 0.02247784896114171
electron:	 0.03180486823065887


Старая реализация без KBinsDiscretizer

In [ ]:
def make_mask(x, bounds):
    lefts = bounds[:-1]
    lefts[0] -= 0.1
    rights = bounds[1:]
    
    a = np.repeat(x, len(rights)).reshape((-1,len(rights))) <= rights
    b = np.repeat(x, len(lefts)).reshape((-1,len(lefts))) > lefts
    
    return np.where(a & b)[1]

def f(x, y_cols):
    d = {}
    for col in y_cols:
        d[f'{col}_mean'] = x[col].mean()
        d[f'{col}_std'] =  x[col].std()

    return pd.Series(d,list(d.keys()))

class Model:
    
    def __init__(self,n_intervals=4):
        self.n_intervals = n_intervals
        
    def train(self, X, Y):
        self.x_cols = X.columns.tolist()
        self.y_cols = Y.columns.tolist()
        
        data = pd.concat([X.copy(), Y.copy()], axis=1)
        masks = pd.DataFrame(index=X.index, columns=X.columns)
        
        quants = [i/self.n_intervals for i in range(self.n_intervals + 1)]
        bounds =  np.quantile(X, quants, axis=0)
        
        for i,col in enumerate(X):
            masks[col] = make_mask(X[col].values, bounds[:,i]).astype(str)
        
        
        data['mask'] = masks.sum(axis=1)
        
        train = data.groupby('mask').apply(lambda x: f(x, self.y_cols))
        n_bins = pd.unique(data['mask']).shape[0]
        assert n_bins == self.n_intervals**3
        
        self.ohe = OneHotEncoder(sparse= False)
        self.ohe.fit_transform(data['mask'])
        
        self.means = LinearRegression()
        self.stds = LinearRegression()
        
        
        self.means.fit(train[self.x_cols].values, train[[f'{col}_mean' for col in self.y_cols]].values)
        self.stds.fit(train[self.x_cols].values, train[[f'{col}_std' for col in self.y_cols]].values)
        
    def predict(self, X):
        prediction = pd.DataFrame()
        means = self.means.predict(X.values)
        stds = self.stds.predict(X.values)
        pred = np.random.standard_normal()*stds + means
        return pd.DataFrame(pred, columns = self.y_cols)


In [ ]:
m = Model(3)
m.train(X,Y)
y = m.predict(X)